# Wrangling SPIROMICS
---

This notebook carrieds out the wrangling process for the SPIROMICS metabolomics data.

## Input

### Libraries

In [1]:
# To use RCall for the first time, one needs to 
# the location of the R home directory.
firstTimeRCall = false
if firstTimeRCall 
    using Pkg
    ENV["R_HOME"] = "C:/PROGRA~1/R/R-42~1.1" # from R.home() in R
    Pkg.build("RCall")
end     

In [2]:
using DataFrames, CSV, Missings
using FreqTables #, CategoricalArrays
using Statistics

### Ext. Functions

In [3]:
include(joinpath(@__DIR__,"..","..","src","wrangle_utils.jl" ));

### Load data ST001639: SPIROMICS

#### Participants

In [4]:
fileIndividuals = realpath(joinpath(@__DIR__,"..","..","data","raw","SPIROMICS","SPIROMICS_ClinicalCovariates.csv"))
dfIndividuals = CSV.read(fileIndividuals, DataFrame;  delim = ',', missingstring = "NA");
first(dfIndividuals, 3)

Row,SUBJID,SAMPLE_NAME,SITE,GOLD_STAGE_COPD_SEVERITY,GENDER,AGE_DERV_01,RACE,BMI_CM01,CURRENT_SMOKER_V1,SMOKING_PACK_YEARS01,POST_FEV1FVC_DERV,PCT_POST_FEV1_V1,V1_PERCENT_EMPHYSEMA_TOTAL,COPD
,String15,String15,String3,Int64,String7,Int64,String31,Float64,Int64?,Float64,Float64,Float64,Float64?,Int64?
1,CU100084,NJHC-01517,CU,2,Female,64,"Non-Hispanic, White",25.3,0,100.0,0.505651,65.4168,11.56,1
2,CU100103,NJHC-01709,CU,0,Female,59,Black/African American,35.9,0,50.0,0.856898,113.274,missing,0
3,CU100139,NJHC-01708,CU,0,Male,52,Black/African American,39.8,1,25.0,0.781953,102.662,0.09,0


In [5]:
unique(dfIndividuals.GOLD_STAGE_COPD_SEVERITY)

4-element Vector{Int64}:
 2
 0
 3
 4

#### Metabolites References

The metabolites references are split in to 4 files:
- neg_metadata.txt
- polar_metadata.txt
- pos_early_metadata.txt
- pos_late_metadata.txt

In [6]:
# Reference metabolomics
vRefMetaFiles = repeat(["path"],4)

vRefMetaFiles[1] = realpath(joinpath(@__DIR__,"..","..","data","raw","SPIROMICS",
                                  "neg_metadata.txt"))
vRefMetaFiles[2] = realpath(joinpath(@__DIR__,"..","..","data","raw","SPIROMICS",
                                  "polar_metadata.txt"))
vRefMetaFiles[3] = realpath(joinpath(@__DIR__,"..","..","data","raw","SPIROMICS",
                                  "pos_early_metadata.txt"))
# Missing 1st column name: metabolite_name, in pos_polar_metadata.txt   
vRefMetaFiles[4] = realpath(joinpath(@__DIR__,"..","..","data","raw","SPIROMICS",
                                  "pos_late_metadata_revised20220829.txt"))#"pos_polar_metadata.txt"))
prepend(vRefMetaFiles[4], "metabolite_name	") # prepend "metabolite_name  "
vRefMetaFiles[4] = realpath(joinpath(@__DIR__,"..","..","data","raw","SPIROMICS",
                                  "new_pos_late_metadata_revised20220829.txt"))#"new_pos_polar_metadata.txt"))

# Generate dataframe
dfRefMetabo = CSV.read(vRefMetaFiles[1], DataFrame;  delim = '	');

for i in 2:length(vRefMetaFiles)
    dfRefMetabo = vcat(dfRefMetabo, CSV.read(vRefMetaFiles[i], DataFrame;  delim = '	'));
end
# add code to remove prepend new file
# rm(vRefMetaFiles[4])
first(dfRefMetabo, 1)

Row,metabolite_name,SUPER.PATHWAY,SUB.PATHWAY,PLATFORM,RI,MASS,PUBCHEM,CAS,KEGG,HMDB
,String,String,String,String15,Float64,Float64,String15,String31,String31,String31
1,(14 or 15)-methylpalmitate (a17:0 or i17:0),Lipid,"Fatty Acid, Branched",LC/MS Neg,5695.0,269.249,NA,NA,NA,NA


In [7]:
names(dfRefMetabo)

10-element Vector{String}:
 "metabolite_name"
 "SUPER.PATHWAY"
 "SUB.PATHWAY"
 "PLATFORM"
 "RI"
 "MASS"
 "PUBCHEM"
 "CAS"
 "KEGG"
 "HMDB"

#### Negative

**Notes:** *The metabolics files do not contains a column header for the name of the metabolites; we need to add a column name.*

In [8]:
fileNegMetabo = realpath(joinpath(@__DIR__,"..","..","data","raw","SPIROMICS","neg.txt"))
prepend(fileNegMetabo, "metabolite_name	")
fileNegMetabo = realpath(joinpath(@__DIR__,"..","..","data","raw","SPIROMICS","new_neg.txt"))
dfNegMetabo = CSV.read(fileNegMetabo, DataFrame;  delim = '	', missingstring = "NA");
rm(fileNegMetabo);
first(dfNegMetabo, 3)

Row,metabolite_name,NJHC-01750,NJHC-01642,NJHC-01498,NJHC-01614,NJHC-01591,NJHC-01819,NJHC-01503,NJHC-01474,NJHC-02018,NJHC-01638,NJHC-01980,NJHC-01558,NJHC-01689,NJHC-01660,NJHC-01419,NJHC-02027,NJHC-01877,NJHC-01956,NJHC-01683,NJHC-01847,NJHC-01499,NJHC-01835,NJHC-01624,NJHC-01788,NJHC-02034,NJHC-01954,NJHC-01987,NJHC-01488,NJHC-01981,NJHC-01961,NJHC-01862,NJHC-01525,NJHC-01572,NJHC-01695,NJHC-01526,NJHC-01595,NJHC-01783,NJHC-01500,NJHC-01449,NJHC-01436,NJHC-01554,NJHC-01626,NJHC-02037,NJHC-01601,NJHC-01963,NJHC-01468,NJHC-01957,NJHC-01496,NJHC-01780,NJHC-01475,NJHC-01742,NJHC-02045,NJHC-01824,NJHC-01476,NJHC-01922,NJHC-01596,NJHC-01477,NJHC-01721,NJHC-01825,NJHC-01511,NJHC-01615,NJHC-02041,NJHC-01900,NJHC-01429,NJHC-01481,NJHC-01727,NJHC-02019,NJHC-01712,NJHC-01982,NJHC-01714,NJHC-01734,NJHC-01866,NJHC-01690,NJHC-01655,NJHC-01650,NJHC-01602,NJHC-02010,NJHC-01751,NJHC-01644,NJHC-01809,NJHC-01691,NJHC-01992,NJHC-01867,NJHC-01842,NJHC-01793,NJHC-01917,NJHC-01918,NJHC-01518,NJHC-01702,NJHC-01735,NJHC-01814,NJHC-01587,NJHC-01858,NJHC-01713,NJHC-01848,NJHC-01708,NJHC-01546,NJHC-01684,NJHC-01863,⋯
,String,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,⋯
1,(14 or 15)-methylpalmitate (a17:0 or i17:0),2.04967e7,1.14421e7,1.13891e7,1.05902e7,5.17257e6,1.19961e7,1.41875e7,1.31663e7,1.78036e7,1.58556e7,9.05915e6,1.87014e7,7.30848e6,1.41185e7,1.20606e7,1.0925e7,1.41611e7,3.48075e7,8.31619e6,1.14378e7,8.84946e6,1.14561e7,9.81313e6,4.49032e6,1.17826e7,8.16998e6,9.10768e6,1.83888e7,5.97925e6,8.3762e6,1.25741e7,1.51996e7,7.0036e6,1.19312e7,7.7332e6,1.05327e7,1.52645e7,1.27435e7,3.61055e6,1.83688e7,1.24218e7,1.03255e7,1.41613e7,1.28019e7,2.42632e7,2.9711e7,9.87941e6,4.93955e6,1.56326e7,1.57483e7,4.45249e6,1.84442e7,1.55161e7,1.59138e7,2.92429e7,6.69368e6,2.10664e7,5.27104e6,1.60187e7,2.27276e6,1.3178e7,1.16773e7,1.01719e7,3.74659e7,1.28217e7,5.22434e6,1.32198e7,5.11602e6,1.05688e7,6.6096e6,1.22091e7,4.54059e6,1.21314e7,3.44984e7,3.35197e7,9.33842e6,6.41717e7,3.24214e7,1.03291e7,1.91197e7,1.36034e7,1.49447e7,715058.0,1.41605e7,1.02259e7,2.48302e7,1.01711e7,1.29584e7,1.3701e7,2.87024e7,1.32356e7,1.71166e7,2.21985e7,2.08107e7,3.1857e6,1.13042e7,4.02623e7,9.75738e6,1.14684e7,⋯
2,(16 or 17)-methylstearate (a19:0 or i19:0),1.13911e6,8.90218e5,8.87839e5,1.12957e6,6.52188e5,1.26706e6,1.0101e6,9.17362e5,1.16149e6,1.1417e6,7.58177e5,1.52422e6,8.60058e5,1.1867e6,6.62986e5,9.86238e5,1.0833e6,1.7691e6,6.79138e5,1.11714e6,785624.0,1.10906e6,8.17624e5,570630.0,7.05474e5,8.13963e5,8.21974e5,1.13621e6,7.23787e5,1.08012e6,8.54601e5,1.03247e6,6.50186e5,1.18514e6,6.03164e5,6.94159e5,7.87944e5,8.20689e5,461979.0,1.41896e6,1.25379e6,7.92101e5,9.26252e5,1.05874e6,1.83668e6,1.77385e6,6.47049e5,4.77353e5,2.09241e6,1.04134e6,3.8681e5,1.05964e6,1.56745e6,1.07932e6,2.01147e6,6.56937e5,1.22901e6,5.39318e5,1.08573e6,2.3795e5,7.48687e5,1.24713e6,1.16056e6,2.33723e6,1.02125e6,5.01623e5,8.53685e5,5.75013e5,1.29312e6,5.19488e5,1.11039e6,5.83797e5,1.04456e6,1.93028e6,1.90461e6,8.6067e5,6.64081e6,2.67044e6,9.39385e5,1.05446e6,1.48197e6,1.62355e6,89914.1,1.09925e6,9.09107e5,1.6843e6,8.68922e5,798356.0,1.12179e6,2.6497e6,1.40648e6,1.35607e6,1.37585e6,1

#### Polar

In [9]:
filePolarMetabo = realpath(joinpath(@__DIR__,"..","..","data","raw","SPIROMICS","polar.txt"))
prepend(filePolarMetabo, "metabolite_name	")
filePolarMetabo = realpath(joinpath(@__DIR__,"..","..","data","raw","SPIROMICS","new_polar.txt"))
dfPolarMetabo = CSV.read(filePolarMetabo, DataFrame;  delim = '	', missingstring = "NA");
rm(filePolarMetabo);
first(dfPolarMetabo, 3)

Row,metabolite_name,NJHC-01750,NJHC-01642,NJHC-01498,NJHC-01614,NJHC-01591,NJHC-01819,NJHC-01503,NJHC-01474,NJHC-02018,NJHC-01638,NJHC-01980,NJHC-01558,NJHC-01689,NJHC-01660,NJHC-01419,NJHC-02027,NJHC-01877,NJHC-01956,NJHC-01683,NJHC-01847,NJHC-01499,NJHC-01835,NJHC-01624,NJHC-01788,NJHC-02034,NJHC-01954,NJHC-01987,NJHC-01488,NJHC-01981,NJHC-01961,NJHC-01862,NJHC-01525,NJHC-01572,NJHC-01695,NJHC-01526,NJHC-01595,NJHC-01783,NJHC-01500,NJHC-01449,NJHC-01436,NJHC-01554,NJHC-01626,NJHC-02037,NJHC-01601,NJHC-01963,NJHC-01468,NJHC-01957,NJHC-01496,NJHC-01780,NJHC-01475,NJHC-01742,NJHC-02045,NJHC-01824,NJHC-01476,NJHC-01922,NJHC-01596,NJHC-01477,NJHC-01721,NJHC-01825,NJHC-01511,NJHC-01615,NJHC-02041,NJHC-01900,NJHC-01429,NJHC-01481,NJHC-01727,NJHC-02019,NJHC-01712,NJHC-01982,NJHC-01714,NJHC-01734,NJHC-01866,NJHC-01690,NJHC-01655,NJHC-01650,NJHC-01602,NJHC-02010,NJHC-01751,NJHC-01644,NJHC-01809,NJHC-01691,NJHC-01992,NJHC-01867,NJHC-01842,NJHC-01793,NJHC-01917,NJHC-01918,NJHC-01518,NJHC-01702,NJHC-01735,NJHC-01814,NJHC-01587,NJHC-01858,NJHC-01713,NJHC-01848,NJHC-01708,NJHC-01546,NJHC-01684,NJHC-01863,⋯
,String,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,⋯
1,"1,6-anhydroglucose",missing,9.29474e5,2.06381e5,3.35506e5,4.29031e5,2.57645e5,missing,8.27625e5,562816.0,4.59934e5,9.7461e5,3.33062e5,6.97769e6,2.15727e5,6.17225e5,3.98595e5,5.91158e5,1.15237e5,1.07949e5,2.4747e6,1.38512e6,4.07972e5,1.89541e6,3.45335e5,4.39639e5,3.38128e5,225458.0,1.01216e6,2.58777e5,703230.0,1.67271e6,2.93784e5,6.26738e5,5.94113e5,3.71473e6,4.62359e5,4.18209e5,91992.1,8.98713e5,3.552e5,3.9023e6,4.55479e6,9.89711e5,missing,1.749e5,1.28825e5,8.93151e5,missing,9.78708e5,8.33946e5,2.37247e6,2.45723e5,2.1841e6,3.96077e5,5.58487e5,7.22791e5,1.38926e5,3.20968e5,1.6518e6,6.03797e6,missing,2.44667e6,8.71531e5,2.76587e5,missing,missing,2.92488e5,8.91329e5,1.76244e6,5.24718e5,1.40339e5,4.36642e5,3.42341e5,2.47597e5,2.74116e5,1.19939e5,1.75291e5,5.19755e5,3.26051e5,2.04354e5,2.14616e6,7.04057e5,1.5887e8,1.08557e6,5.45389e5,missing,1.13757e6,5.93073e5,5.75359e5,3.07595e5,3.16316e5,missing,4.78068e5,2.11117e5,2.65191e6,1.56563e5,6.11326e5,1.96252e5,6.49898e5,⋯
2,"2,3-dihydroxyisovalerate",2.54964e5,2.25658e5,3.16226e5,4.9566e5,1.25176e5,1.16234e5,1.25729e5,3.58954e5,2.18205e5,5.27613e6,2.93558e5,3.67141e5,1.98116e5,1.61752e5,87329.0,4.25065e5,1.75222e5,8.33208e5,630459.0,2.28936e5,2.34653e5,missing,missing,1.01476e5,1.62089e5,2.10798e5,1.06451e6,1.12709e5,4.80242e5,2.3184e5,1.41269e6,2.9957e6,1.13289e5,367571.0,missing,51873.5,1.90045e6,2.31866e5,2.80902e5,3.21315e6,1.75141e6,3.3388e5,9.05364e5,1.99062e5,6.81961e6,2.76428e6,236918.0,1.36172e6,8.74725e5,3.02785e5,1.31277e6,2.37911e5,missing,54414.8,3.45843e5,1.14306e5,2.3153e5,3.21021e5,4.3315e5,3.83082e5,4.80588e5,3.64311e5,2.97309e5,1.00711e6,43357.3,1.37622e5,2.10496e5,6.32033e6,2.71839e5,4.4184e5,1.38677e6,142827.0,missing,85247.2,71135.2,missing,missing,9.00506e6,missing,missing,4.91385e6,77368.4,7.91784e7,1.16198e6,4.21665e5,missing,1.15252e5,70244.1,1.11018e6,3.29128e6,1.28105e5,132578.0,missing,2.00994e6,5.40189e6,57753.0,3.54066e5,7.42361e5,4.85621e6,⋯
3,"2,4,6-trihydroxybenzoate",missing,

#### Positive early

In [10]:
filePosEarlyMetabo = realpath(joinpath(@__DIR__,"..","..","data","raw","SPIROMICS","pos_early.txt"))
prepend(filePosEarlyMetabo, "metabolite_name	")
filePosEarlyMetabo = realpath(joinpath(@__DIR__,"..","..","data","raw","SPIROMICS","new_pos_early.txt"))
dfPosEarlyMetabo = CSV.read(filePosEarlyMetabo, DataFrame;  delim = '	', missingstring = "NA");
rm(filePosEarlyMetabo)
first(dfPosEarlyMetabo, 3)

Row,metabolite_name,NJHC-01750,NJHC-01642,NJHC-01498,NJHC-01614,NJHC-01591,NJHC-01819,NJHC-01503,NJHC-01474,NJHC-02018,NJHC-01638,NJHC-01980,NJHC-01558,NJHC-01689,NJHC-01660,NJHC-01419,NJHC-02027,NJHC-01877,NJHC-01956,NJHC-01683,NJHC-01847,NJHC-01499,NJHC-01835,NJHC-01624,NJHC-01788,NJHC-02034,NJHC-01954,NJHC-01987,NJHC-01488,NJHC-01981,NJHC-01961,NJHC-01862,NJHC-01525,NJHC-01572,NJHC-01695,NJHC-01526,NJHC-01595,NJHC-01783,NJHC-01500,NJHC-01449,NJHC-01436,NJHC-01554,NJHC-01626,NJHC-02037,NJHC-01601,NJHC-01963,NJHC-01468,NJHC-01957,NJHC-01496,NJHC-01780,NJHC-01475,NJHC-01742,NJHC-02045,NJHC-01824,NJHC-01476,NJHC-01922,NJHC-01596,NJHC-01477,NJHC-01721,NJHC-01825,NJHC-01511,NJHC-01615,NJHC-02041,NJHC-01900,NJHC-01429,NJHC-01481,NJHC-01727,NJHC-02019,NJHC-01712,NJHC-01982,NJHC-01714,NJHC-01734,NJHC-01866,NJHC-01690,NJHC-01655,NJHC-01650,NJHC-01602,NJHC-02010,NJHC-01751,NJHC-01644,NJHC-01809,NJHC-01691,NJHC-01992,NJHC-01867,NJHC-01842,NJHC-01793,NJHC-01917,NJHC-01918,NJHC-01518,NJHC-01702,NJHC-01735,NJHC-01814,NJHC-01587,NJHC-01858,NJHC-01713,NJHC-01848,NJHC-01708,NJHC-01546,NJHC-01684,NJHC-01863,⋯
,String,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,⋯
1,(2-butoxyethoxy)acetic acid,missing,missing,missing,missing,missing,missing,9.49767e5,missing,1.16312e6,missing,1.1836e6,missing,missing,missing,72623.5,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,1.16846e6,2.55219e5,missing,4.39724e6,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,1.15575e6,missing,missing,1.6912e5,1.05078e5,missing,3.0053e5,missing,3.82737e5,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,5.89784e5,missing,missing,missing,missing,missing,missing,missing,914859.0,missing,1.87053e6,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,4.77088e5,missing,missing,2.23564e5,⋯
2,(R)-3-hydroxybutyrylcarnitine,1.86193e6,3.39531e6,6.48398e5,4.83569e6,8.05834e5,9.12263e6,4.64522e6,2.63045e6,1.29683e6,1.83508e6,1.29373e6,3.57467e6,8.08166e5,6.29753e6,1.56593e6,2.48744e6,3.72404e6,5.19554e6,1.09376e6,3.85654e5,1.3313e6,2.12037e6,missing,7.20039e5,2.35867e6,6.70331e5,2.04467e6,1.51139e6,1.01274e6,1.13657e6,2.17752e6,1.47757e6,3.05927e6,4.8921e6,8.67714e5,7.18352e5,8.18912e5,3.40021e6,7.03703e5,3.00834e6,1.12634e6,2.79669e6,1.664e6,6.08884e5,3.41361e6,3.00974e6,8.6269e5,2.02432e6,2.43637e6,4.30831e6,2.08189e6,1.21956e6,5.2051e6,3.17288e6,2.69927e6,missing,2.99313e6,2.86818e6,missing,missing,3.75663e6,4.23368e6,3.15263e6,5.99724e6,1.09532e6,6.5606e5,1.16126e6,4.79851e6,1.11678e7,1.43682e6,5.19377e6,missing,1.54094e6,1.31569e7,1.03087e7,5.06521e6,4.50332e6,2.44661e6,1.61013e6,3.78282e6,1.89369e6,4.44759e6,2.90869e6,1.78602e6,1.86279e6,1.67995e7,2.4507e6,2.6297e6,1.65452e6,5.81066e6,1.04056e6,9.07086e6,1.38013e6,7.1502e6,missing,1.14082e6,4.38609e6,2.65214e6,3.14531e6,⋯
3,(S)-3-hydroxybutyrylcarnitine,1.3686e6,2.81463e6,1.22605e6,2.42628e6,5.98297e5,2.16304e6,2.51603e6,1.57787e6,1.96427e6,1.2477e6,7.62

#### Positive late

In [11]:
filePosLateMetabo = realpath(joinpath(@__DIR__,"..","..","data","raw","SPIROMICS","pos_late.txt"))
prepend(filePosLateMetabo, "metabolite_name	")
filePosLateMetabo = realpath(joinpath(@__DIR__,"..","..","data","raw","SPIROMICS","new_pos_late.txt"))
dfPosLateMetabo = CSV.read(filePosLateMetabo, DataFrame;  delim = '	', missingstring = "NA");
rm(filePosLateMetabo)
first(dfPosLateMetabo, 3)

Row,metabolite_name,NJHC-01750,NJHC-01642,NJHC-01498,NJHC-01614,NJHC-01591,NJHC-01819,NJHC-01503,NJHC-01474,NJHC-02018,NJHC-01638,NJHC-01980,NJHC-01558,NJHC-01689,NJHC-01660,NJHC-01419,NJHC-02027,NJHC-01877,NJHC-01956,NJHC-01683,NJHC-01847,NJHC-01499,NJHC-01835,NJHC-01624,NJHC-01788,NJHC-02034,NJHC-01954,NJHC-01987,NJHC-01488,NJHC-01981,NJHC-01961,NJHC-01862,NJHC-01525,NJHC-01572,NJHC-01695,NJHC-01526,NJHC-01595,NJHC-01783,NJHC-01500,NJHC-01449,NJHC-01436,NJHC-01554,NJHC-01626,NJHC-02037,NJHC-01601,NJHC-01963,NJHC-01468,NJHC-01957,NJHC-01496,NJHC-01780,NJHC-01475,NJHC-01742,NJHC-02045,NJHC-01824,NJHC-01476,NJHC-01922,NJHC-01596,NJHC-01477,NJHC-01721,NJHC-01825,NJHC-01511,NJHC-01615,NJHC-02041,NJHC-01900,NJHC-01429,NJHC-01481,NJHC-01727,NJHC-02019,NJHC-01712,NJHC-01982,NJHC-01714,NJHC-01734,NJHC-01866,NJHC-01690,NJHC-01655,NJHC-01650,NJHC-01602,NJHC-02010,NJHC-01751,NJHC-01644,NJHC-01809,NJHC-01691,NJHC-01992,NJHC-01867,NJHC-01842,NJHC-01793,NJHC-01917,NJHC-01918,NJHC-01518,NJHC-01702,NJHC-01735,NJHC-01814,NJHC-01587,NJHC-01858,NJHC-01713,NJHC-01848,NJHC-01708,NJHC-01546,NJHC-01684,NJHC-01863,⋯
,String,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,⋯
1,"1,2-dilinoleoyl-GPC (18:2/18:2)",8.35689e7,1.31306e8,1.06189e8,1.13068e8,1.20376e8,3.98275e7,1.66163e8,1.2571e8,8.90618e7,8.74958e7,1.41083e8,6.46712e7,9.02342e7,9.46391e7,8.70244e7,1.54663e8,8.06085e7,5.01139e7,1.08296e8,2.08461e8,1.04895e8,7.06158e7,6.9857e7,5.95349e7,6.62442e7,1.48552e8,1.50581e8,1.73074e8,1.73926e8,1.40051e8,1.30224e8,1.62944e8,1.03421e8,1.33324e8,1.10092e8,1.07996e8,9.47514e7,8.22368e7,7.08013e7,1.26952e8,1.79759e8,1.12226e8,2.1503e8,1.20185e8,9.4417e7,7.45765e7,9.92566e7,7.46135e7,1.07516e8,1.47631e8,5.1442e7,1.30825e8,1.32504e8,1.09794e8,1.47514e8,1.30124e8,1.40696e8,4.07877e7,1.61328e8,2.233e8,1.32877e8,1.2316e8,8.13447e7,1.31285e8,1.10012e8,8.08064e7,1.70297e8,1.65427e8,8.98718e7,8.6346e7,9.9223e7,1.08939e8,1.2504e8,6.90145e7,8.94263e7,8.79441e7,9.91764e7,6.75574e7,1.01397e8,6.8174e7,1.33963e8,1.06085e8,4.25157e5,1.3665e8,1.21375e8,6.00597e7,4.03569e7,8.09244e7,2.5675e8,1.03554e8,1.26027e8,8.57805e7,1.50228e8,8.6356e7,1.21484e8,4.31782e7,1.05889e8,1.105e8,1.24835e8,⋯
2,"1,2-dipalmitoyl-GPC (16:0/16:0)",5.01226e7,3.41086e7,5.54832e7,5.73641e7,4.54575e7,4.37259e7,4.86805e7,4.95447e7,4.4005e7,5.46287e7,4.1113e7,5.80674e7,5.65275e7,4.99471e7,4.13048e7,4.45523e7,4.11628e7,4.3878e7,4.55188e7,5.28457e7,4.9745e7,3.95083e7,2.44572e7,4.78726e7,3.76249e7,4.62412e7,5.82358e7,4.31937e7,2.96737e7,4.64424e7,4.80161e7,6.41809e7,4.23205e7,4.15771e7,3.76538e7,5.15789e7,4.78193e7,3.1023e7,3.56475e7,5.26665e7,5.31019e7,2.87438e7,4.67286e7,4.23172e7,5.04956e7,3.477e7,4.16583e7,3.76631e7,4.90196e7,4.68911e7,4.30872e7,4.47355e7,4.19223e7,3.92229e7,5.18078e7,3.86951e7,4.01721e7,4.49484e7,5.57127e7,5.81796e7,4.74813e7,3.98441e7,3.44949e7,4.2536e7,4.39773e7,3.31457e7,4.77307e7,5.17834e7,5.79261e7,4.08773e7,4.76285e7,4.09656e7,3.72912e7,4.37428e7,4.83809e7,5.4802e7,6.37133e7,4.33435e7,3.99776e7,4.26438e7,4.08971e7,3.31931e7,3.92537e5,4.27941e7,4.20491e7,3.82075e7,3.67731e7,3.66322e7,5.94059e7,5.05027e7,5.63094e7,4.54577e7,4.66429e7,4.58161e7,4.77072e7,

#### Check that each dataframe contains the same column names(sample names). 

In [12]:
names(dfNegMetabo) == names(dfPolarMetabo) == names(dfPosEarlyMetabo) == names(dfPosLateMetabo)

true

## ST001639: SPIROMICS exploration

### Individuals

In [13]:
println("The participants dataset contains $(size(dfIndividuals, 1)) individuals and $(size(dfIndividuals, 2)) covariates.")

The participants dataset contains 447 individuals and 14 covariates.


In [14]:
describe(dfIndividuals)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,SUBJID,,CU100084,,WF125434,0,String15
2,SAMPLE_NAME,,NJHC-01397,,NJHC-02046,0,String15
3,SITE,,CU,,WF,0,String3
4,GOLD_STAGE_COPD_SEVERITY,1.28412,0,2.0,4,0,Int64
5,GENDER,,Female,,Male,0,String7
6,AGE_DERV_01,63.5168,40,65.0,80,0,Int64
7,RACE,,Black/African American,,"Non-Hispanic, White",0,String31
8,BMI_CM01,28.4996,14.5,28.3,39.8,0,Float64
9,CURRENT_SMOKER_V1,0.362613,0,0.0,1,3,"Union{Missing, Int64}"


Notes:`ATS_PackYears`, i.e. smoking number of pack-years, and `Insp_LAA950_total_Thirona`, i.e. percent emphysema, contain `NA`.

Check how many `NA`:

In [15]:
vMissing = map(eachcol(dfIndividuals)) do col
               sum(ismissing.(col))
           end
idxColMiss = findall(vMissing .!= 0)
for i in idxColMiss
    println("$(names(dfIndividuals)[i]) contains $(vMissing[i]) missing values.")
end

CURRENT_SMOKER_V1 contains 3 missing values.
V1_PERCENT_EMPHYSEMA_TOTAL contains 63 missing values.
COPD contains 11 missing values.


#### Change variable names: 

In [16]:
rename!(dfIndividuals, Dict(:SUBJID => "SampleName", :SAMPLE_NAME => "SampleID",
                            :SITE => "Site", :GOLD_STAGE_COPD_SEVERITY => "FinalGold",
                            :GENDER => "Sex", :AGE_DERV_01 => "Age", :RACE => "race",
                            :SMOKING_PACK_YEARS01 => "SmokingPackYears", :BMI_CM01 => "BMI",
                            :POST_FEV1FVC_DERV => "FEV1_FVC", :PCT_POST_FEV1_V1 => "FEV1pp",
                            :CURRENT_SMOKER_V1 => "CurrentSmoker",
                            :V1_PERCENT_EMPHYSEMA_TOTAL => "PercentEmphysema"));

Verify how many factors per categorical variables, *i.e.* gender, race, smoking status, COPD case status.

In [17]:
vCovariateNames = [:Sex, :race, :CurrentSmoker, :COPD]
vUniqueCat = map(eachcol(dfIndividuals[:, vCovariateNames])) do col
                 join(unique(col), ",", " and ")
             end
for i in 1:length(vUniqueCat)
    println("$(string(vCovariateNames[i])) variable contains: $(vUniqueCat[i]) values.")
end

Sex variable contains: Female and Male values.
race variable contains: Non-Hispanic, White and Black/African American values.
CurrentSmoker variable contains: 0,1 and missing values.
COPD variable contains: 1,0 and missing values.


Convert:
- `race` into `NHW`, where the value *1* corresponds to non-Hispanic White and *0* otherwise.
- `smoking_status` into `CurrentSmoker`, , where the value *1* corresponds to Current Smoker and *0* to Former Smoker.

In [18]:
#  non-Hipanic White
vNHW = zeros(Int, size(dfIndividuals, 1));
idxNHW = findall(dfIndividuals.race .== "Non-Hispanic, White");
vNHW[idxNHW] .= 1
dfIndividuals.NHW = (vNHW);

In [19]:
# Drop `race` and `smoking_status`
select!(dfIndividuals, Not([:race]));

Get demographics SPIROMICS cohort by sex.

In [20]:
# Group by sex
gdf = groupby(dfIndividuals, :Sex);

# Get mean values
mymean(X) = mean(skipmissing(X))
df1a = combine(gdf, [:Age, :BMI, :SmokingPackYears, :PercentEmphysema] .=> mymean)
df1a[:,2:end] = round.((df1a[:,2:end]); digits = 1)
rename!(df1a, Dict(:Age_mymean => "Age", :BMI_mymean => "BMI",
                   :SmokingPackYears_mymean => "SmokingPackYears", :PercentEmphysema_mymean => "PercentEmphysema"));

# Get standard deviation values
mystd(X) = std(skipmissing(X))
df1b = combine(gdf, [:Age, :BMI, :SmokingPackYears, :PercentEmphysema] .=> mystd)
df1b[:,2:end] = round.((df1b[:,2:end]); digits = 1)
rename!(df1b, Dict(:Age_mystd => "Age", :BMI_mystd => "BMI",
                   :SmokingPackYears_mystd => "SmokingPackYears", :PercentEmphysema_mystd => "PercentEmphysema"));

# Join mean and standard deviation values
dfDem1 = string.(df1a[:,2:end]).*repeat(["("], size(df1a,1),size(df1a,2)-1).* 
         string.(df1b[:,2:end]).*repeat([")"], size(df1a,1),size(df1a,2)-1);
insertcols!(dfDem1, 1, :Sex => df1a.Sex, :Participants => combine(gdf, nrow)[:,2])

# Get sum values
sum_skipmissing(x)= sum(skipmissing(x));
df2a = combine(gdf, [:NHW, :CurrentSmoker, :COPD] .=> sum_skipmissing)

# Get number of participant excluding those with missing values
nrow_skipmissing(x)= length(collect(skipmissing(x)));
dfParticipantSkipmissing = combine(gdf, [:NHW, :CurrentSmoker, :COPD] .=> nrow_skipmissing)


# Get percentage values
df2b = round.((df2a[:,2:end]./ Matrix(dfParticipantSkipmissing[:, 2:end])).*100, digits = 1)

# Join sum and percentage values
dfDem2 = string.(df2a[:,2:end]).*repeat(["("], size(df2a,1),size(df2a,2)-1).* 
         string.(df2b[:,1:end]).*repeat([")"], size(df2a,1),size(df2a,2)-1)
insertcols!(dfDem2, 1, :Sex => df2a.Sex)

rename!(dfDem2, Dict(:NHW_sum_skipmissing => "NHW", :CurrentSmoker_sum_skipmissing => "CurrentSmoker",
                   :COPD_sum_skipmissing => "COPD"))
# Join demographics dataframes
dfDem = leftjoin(dfDem1, dfDem2, on = :Sex )

# Pivot table
dfDem = permutedims(dfDem, 1, "Variable")

Row,Variable,Female,Male
,String,Any,Any
1,Participants,214,233
2,Age,63.0(8.8),64.0(8.0)
3,BMI,28.4(5.6),28.6(4.9)
4,SmokingPackYears,45.5(21.3),55.5(38.1)
5,PercentEmphysema,5.0(8.8),5.3(9.1)
6,NHW,163(76.2),191(82.0)
7,CurrentSmoker,70(32.9),91(39.4)
8,COPD,102(49.5),140(60.9)


The demographic table is identical to the article "*Metabolomic Profiling Reveals Sex Specific Associations with
Chronic Obstructive Pulmonary Disease and Emphysema*"(2021).

#### Save processed individuals dataset:

In [21]:
first(dfIndividuals)

Row,SampleName,SampleID,Site,FinalGold,Sex,Age,BMI,CurrentSmoker,SmokingPackYears,FEV1_FVC,FEV1pp,PercentEmphysema,COPD,NHW
,String15,String15,String3,Int64,String7,Int64,Float64,Int64?,Float64,Float64,Float64,Float64?,Int64?,Int64
1,CU100084,NJHC-01517,CU,2,Female,64,25.3,0,100.0,0.505651,65.4168,11.56,1,1


In [22]:
fileIndividuals = joinpath(@__DIR__,"..","..","data","processed","SPIROMICS","SPIROMICS_ClinicalCovariates.csv");
dfIndividuals |> CSV.write(fileIndividuals);

### Metabolomics References

#### Create dataframe whith pathways

In [23]:
names(dfRefMetabo)

10-element Vector{String}:
 "metabolite_name"
 "SUPER.PATHWAY"
 "SUB.PATHWAY"
 "PLATFORM"
 "RI"
 "MASS"
 "PUBCHEM"
 "CAS"
 "KEGG"
 "HMDB"

Since the [PUBCHEM ID list](https://www.metabolomicsworkbench.org/rest/study/study_id/ST001639/metabolites/text) is incomplete, we need to generate a pseudo CompID. First sort by metabolite name then generate the IDs.

In [24]:
sort!(dfRefMetabo, :metabolite_name)
insertcols!(dfRefMetabo, 2, :CompID => replace.(string.(collect(1:size(dfRefMetabo, 1)).+99990000), "9999"=>"comp"));

In [25]:
first(dfRefMetabo, 3)

Row,metabolite_name,CompID,SUPER.PATHWAY,SUB.PATHWAY,PLATFORM,RI,MASS,PUBCHEM,CAS,KEGG,HMDB
,String,String,String,String,String15,Float64,Float64,String15,String31,String31,String31
1,(14 or 15)-methylpalmitate (a17:0 or i17:0),comp0001,Lipid,"Fatty Acid, Branched",LC/MS Neg,5695.0,269.249,NA,NA,NA,NA
2,(16 or 17)-methylstearate (a19:0 or i19:0),comp0002,Lipid,"Fatty Acid, Branched",LC/MS Neg,5993.0,297.28,3083779,2724-59-6,NA,HMDB37397
3,(2 or 3)-decenoate (10:1n7 or n8),comp0003,Lipid,Medium Chain Fatty Acid,LC/MS Neg,4990.0,169.123,NA,NA,NA,NA


Keep metabolite_name, comp ID, super pathways and sub pathways.

In [26]:
first(dfRefMetabo[:, Symbol.(["metabolite_name", "CompID", "SUPER.PATHWAY", "SUB.PATHWAY"])], 3)

Row,metabolite_name,CompID,SUPER.PATHWAY,SUB.PATHWAY
,String,String,String,String
1,(14 or 15)-methylpalmitate (a17:0 or i17:0),comp0001,Lipid,"Fatty Acid, Branched"
2,(16 or 17)-methylstearate (a19:0 or i19:0),comp0002,Lipid,"Fatty Acid, Branched"
3,(2 or 3)-decenoate (10:1n7 or n8),comp0003,Lipid,Medium Chain Fatty Acid


In [27]:
last(dfRefMetabo[:, Symbol.(["metabolite_name", "CompID", "SUPER.PATHWAY", "SUB.PATHWAY"])], 3)

Row,metabolite_name,CompID,SUPER.PATHWAY,SUB.PATHWAY
,String,String,String,String
1,ximenoylcarnitine (C26:1)*,comp1172,Lipid,"Fatty Acid Metabolism (Acyl Carnitine, Monounsaturated)"
2,xylose,comp1173,Carbohydrate,Pentose Metabolism
3,zolpidem,comp1174,Xenobiotics,Drug - Psychoactive


In [28]:
# Select variables of interest and rename accordingly
rename!(dfRefMetabo, Dict(Symbol("SUB.PATHWAY") => "SubPathway", Symbol("SUPER.PATHWAY") => "SuperPathway")) 
select!(dfRefMetabo, [:metabolite_name, :CompID, :SubPathway, :SuperPathway]);

In [29]:
# Create 2 new variables name SubClassID and SuperClassID that 
# contain a codification of pathways

# Group by Super Pathway
gdf = groupby(dfRefMetabo, :SuperPathway);

nTotalSub = length(unique(dfRefMetabo.SubPathway))
vInit = repeat(["NA"], nTotalSub);
dfNewRef = DataFrame(SubPathway = vInit, SubClassID = vInit,
                     SuperPathway = vInit, SuperClassID = vInit);

In [30]:
# Generate pathway ID references for the metabolites
idxStart = 1

for i in 1:(length(gdf)-1)
    vSub = unique(gdf[i].SubPathway)
    nSub = length(vSub)
    
    idxEnd = idxStart + nSub - 1
    
    dfNewRef.SubPathway[idxStart:idxEnd] = vSub;
    dfNewRef.SubClassID[idxStart:idxEnd] = uppercase(gdf[i].SuperPathway[1][1:3]).*string.(collect(1:nSub));
    dfNewRef.SuperPathway[idxStart:idxEnd] .= gdf[i].SuperPathway[1];
    dfNewRef.SuperClassID[idxStart:idxEnd] .= uppercase(gdf[i].SuperPathway[1][1:3]);
    
    idxStart = idxEnd + 1
end

In [31]:
# Initiatlize vector
nMeta = size(dfRefMetabo, 1);
vSubClass = repeat(["NA"], nMeta);
vSupClass = repeat(["NA"], nMeta);

for i in 1:length(dfNewRef.SubPathway)
    idx = findall(dfRefMetabo.SubPathway.== dfNewRef.SubPathway[i])
    vSubClass[idx] .= dfNewRef.SubClassID[i]
    vSupClass[idx] .= dfNewRef.SuperClassID[i]
end
dfRefMetabo.SubClassID = vSubClass; 
dfRefMetabo.SuperClassID = vSupClass;


# Insert 0 in SubID when ID number less than 10. It helps for sorting.
idxSub2Change = findall(length.(dfRefMetabo.SubClassID) .== 4)
for i in 1:length(idxSub2Change) 
    dfRefMetabo.SubClassID[idxSub2Change[i]] = dfRefMetabo.SubClassID[idxSub2Change[i]][1:3]*"0"*dfRefMetabo.SubClassID[idxSub2Change[i]][4]
end

#### Check cotinine bio chemical

The cotinine levels will be imputed differently if missing is more than 20%.

In [32]:
# check for cotinine
idxCotinine = findall(occursin.(r"(?i)cotinine", dfRefMetabo.metabolite_name))
dfRefMetabo[idxCotinine, :]

Row,metabolite_name,CompID,SubPathway,SuperPathway,SubClassID,SuperClassID
,String,String,String,String,String,String
1,3-hydroxycotinine glucuronide,comp0216,Tobacco Metabolite,Xenobiotics,XEN09,XEN
2,cotinine,comp0588,Tobacco Metabolite,Xenobiotics,XEN09,XEN
3,cotinine N-oxide,comp0589,Tobacco Metabolite,Xenobiotics,XEN09,XEN
4,hydroxycotinine,comp0808,Tobacco Metabolite,Xenobiotics,XEN09,XEN
5,norcotinine,comp0926,Tobacco Metabolite,Xenobiotics,XEN09,XEN


#### Explore frequency table

In [33]:
freqtable(dfRefMetabo.SuperPathway)

9-element Named Vector{Int64}
Dim1                              │ 
──────────────────────────────────┼────
Amino Acid                        │ 228
Carbohydrate                      │  25
Cofactors and Vitamins            │  43
Energy                            │  11
Lipid                             │ 435
Nucleotide                        │  41
Partially Characterized Molecules │  30
Peptide                           │  43
Xenobiotics                       │ 318

In [34]:
idxLipid = findall(dfRefMetabo.SuperPathway .== "Lipid")
freqtable(dfRefMetabo[idxLipid, :SubPathway]); #  |> show;

#### Save processed individuals dataset:

In [35]:
fileRef = joinpath(@__DIR__,"..","..","data","processed","SPIROMICS","refMeta.csv");
dfRefMetabo |> CSV.write(fileRef);

### Negative

**Notes:** *The metabolics files use the metabolite name to identify each metabolite. We need to use an ID similar to the compound ID from the COPDGenegene cohort. Since compound IDs are not available we use the workbench metabolite IDs.*

Filter `dfNegMetabo` sample according to the individuals dataframe `dfIndividuals`:

#### Keep complete cases

In [36]:
dfNegMetabo = keepComplete(dfNegMetabo, dfIndividuals, dfRefMetabo; sampleCol = :SampleID, metaCol = :metabolite_name );

#### Save filtered sample negative metabolites levels dataset:

In [37]:
fileNeg = joinpath(@__DIR__,"..","..","data","processed","SPIROMICS","negMeta.csv");
dfNegMetabo |> CSV.write(fileNeg);

In [38]:
println("The negative metabolite dataset contains $(size(dfNegMetabo, 2)-1) samples and $(size(dfNegMetabo, 1)) metabolites.")

The negative metabolite dataset contains 372 samples and 588 metabolites.


### Polar

Filter `dfPolarMetabo` sample according to the individuals dataframe `dfIndividuals`:

#### Keep complete cases

In [39]:
dfPolarMetabo = keepComplete(dfPolarMetabo, dfIndividuals, dfRefMetabo; sampleCol = :SampleID, metaCol = :metabolite_name );

#### Save filtered sample polar metabolites levels dataset:

In [40]:
filePolar = joinpath(@__DIR__,"..","..","data","processed","SPIROMICS","polarMeta.csv");
dfPolarMetabo |> CSV.write(filePolar);

In [41]:
println("The polar metabolite dataset contains $(size(dfPolarMetabo, 2)-1) samples and $(size(dfPolarMetabo, 1)) metabolites.")

The polar metabolite dataset contains 372 samples and 96 metabolites.


### Positive Early

Filter `dfPosEarlyMetabo` sample according to the individuals dataframe `dfIndividuals`:

[Polar molecules elute earlier and nonpolar molecules later.](https://www.sciencedirect.com/topics/immunology-and-microbiology/metabolome-analysis)

#### Keep complete cases

In [42]:
dfPosEarlyMetabo = keepComplete(dfPosEarlyMetabo, dfIndividuals, dfRefMetabo; sampleCol = :SampleID, metaCol = :metabolite_name );

#### Save filtered sample positive early metabolites levels dataset:

In [43]:
filePosEarly = joinpath(@__DIR__,"..","..","data","processed","SPIROMICS","posEarlyMeta.csv");
dfPosEarlyMetabo |> CSV.write(filePosEarly);

In [44]:
println("The positive early metabolite dataset contains $(size(dfPosEarlyMetabo, 2)-1) samples and $(size(dfPosEarlyMetabo, 1)) metabolites.")

The positive early metabolite dataset contains 372 samples and 258 metabolites.


### Positive Late

Filter `dfPosLateMetabo` sample according to the individuals dataframe `dfIndividuals`:

[Polar molecules elute earlier and nonpolar molecules later.](https://www.sciencedirect.com/topics/immunology-and-microbiology/metabolome-analysis)

#### Keep complete cases

In [45]:
dfPosLateMetabo = keepComplete(dfPosLateMetabo, dfIndividuals, dfRefMetabo; sampleCol = :SampleID, metaCol = :metabolite_name );

#### Save filtered sample positive late metabolites levels dataset:

In [46]:
filePosLate = joinpath(@__DIR__,"..","..","data","processed","SPIROMICS","posLateMeta.csv");
dfPosLateMetabo |> CSV.write(filePosLate);

In [47]:
println("The positive late metabolite dataset contains $(size(dfPosLateMetabo, 2)-1) samples and $(size(dfPosLateMetabo, 1)) metabolites.")

The positive late metabolite dataset contains 372 samples and 232 metabolites.


In [48]:
names(dfRefMetabo)

6-element Vector{String}:
 "metabolite_name"
 "CompID"
 "SubPathway"
 "SuperPathway"
 "SubClassID"
 "SuperClassID"

In [49]:
# innerjoin(dfRefMetabo[:, [:metabolite_name, :CompID]], dfPosLateMetabo, on = :metabolite_name);

In [50]:
# dfPosLateMetabo.metabolite_name ⊆ dfRefMetabo.metabolite_name

In [51]:
# fileList = joinpath(@__DIR__,"..","..","data","processed","SPIROMICS","list.csv");
# DataFrame(Name = dfPosLateMetabo.metabolite_name) |> CSV.write(fileList);